In [1]:
from sklearn import preprocessing
import numpy as np
import pandas as pd
from collections import defaultdict

In [2]:
#import data
fields = ['toolkit','forks','stars','so_tag_counts','so_question_count','search_results', 'growth_rate','market_cap_value','thirty_day_volume']
DF = pd.read_csv('../output/blockchain_data.csv', usecols=fields)

In [3]:
#replace not a number values with 0
DF.fillna(0, inplace=True)

In [4]:
#scale the values, such that for each metric: mean = 0 and std = 1
metrics = ['forks','stars','so_tag_counts','so_question_count','search_results', 'growth_rate','market_cap_value','thirty_day_volume']
scaled_DF = pd.DataFrame(columns=metrics)

for metric in metrics:
    mean = DF[metric].mean()
    std = DF[metric].std()
    scaled_DF[metric] = DF[metric].apply(lambda x: (x-mean)/std)

scaled_DF['toolkit']= DF['toolkit']

In [5]:
#merge forks and stars to one GitHub metric, Tags and Questions to Stack Overflow, and scale
final_DF = pd.DataFrame(columns=['Toolkit', 'Rank', 'Overall', 'Github', 'Stack Overflow', 'Google Results', 'Market Cap Value', 'Volume Over 30 Days'])
final_DF['Library'] = scaled_DF['toolkit']
final_DF['Github'] = scaled_DF[['forks', 'stars']].mean(axis=1)
final_DF['Stack Overflow'] = scaled_DF[['so_tag_counts', 'so_question_count']].mean(axis=1)
final_DF['Google Results'] = scaled_DF[['search_results', 'growth_rate']].mean(axis=1)

#scale financial metrics
final_DF['Market Cap Value'] = scaled_DF[['market_cap_value']].mean(axis=1)
final_DF['Volume Over 30 Days'] = scaled_DF[['thirty_day_volume']].mean(axis=1)

#compute overall score and rank
final_DF['Overall'] = final_DF[['Github','Stack Overflow','Google Results','Market Cap Value', 'Volume Over 30 Days']].sum(axis=1)
final_DF['Rank'] = final_DF['Overall'].rank(ascending=0).astype(int)

In [6]:
#sort, reindex, and set style for rankings table
final_DF.sort_values(['Overall'], axis=0, ascending=False, inplace=True)
final_DF = final_DF.reindex_axis(['Library','Rank','Overall','Github','Stack Overflow','Google Results','Market Cap Value', 'Volume Over 30 Days'], axis=1)
final_DF.style.set_table_styles([
    {'selector': '.row_heading, .blank', 'props': [('display', 'none;')]}
])

In [7]:
#save rankings table to csv file
final_DF.to_csv('../output/blockchain_Rankings.csv', index = False)